In [19]:
import requests, bs4, nltk.corpus
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from time import sleep
# nltk.download('stopwords')
from nltk.corpus import stopwords

In [20]:
# Misc func.

def print_df(df):
    for i in df:
        print(i)

In [21]:
def get_loc(result):
    try:
        return entry.find('div', {'class': 'companyLocation'}).text
    except Exception as e:
        return 'NA'


def get_comp(result):
    try:
        return result.find('span', {'class': 'companyName'}).text
    except:
        return 'NA'


def get_job(result):
    try:
        return result.find('h2', {'class': 'jobTitle'}).text
    except:
        return 'NA'


def get_sal(result):
    try:
        return result.find('div', {'class': 'salaryOnly'}).text
    except:
        return 'NA'


def get_desc(result):
    try:
        return result.find('div', {'class': 'job-snippet'}).text
    except:
        return 'NA'
def get_rating(result):
    try:
        return result.find('span', {'class': 'ratingNumber'}).text
    except:
        return 'NA'

In [22]:
job_titles = [
    'Data+Scientist', 'full+stack+developer', 'big+data+engineer',
    'software+architect', 'blockchain', 'devops+engineer', 'cloud+architect',
    'artificial+intelligence', 'product+manager'
]

len(job_titles)
results_dic = {
    
}

9

In [34]:
results = []

In [36]:
max_results_per_job = 7000
start_time = datetime.now()
print(start_time)
for job in job_titles:
    try:
        print("---------- Started for " + job)
        for start in range(0, max_results_per_job, 100):
            url = "https://in.indeed.com/jobs?q=" + job + "&start=" + str(
                start) + '&limit=100'
            html = requests.get(url)
            soup = BeautifulSoup(html.text, 'html.parser')
            for result in soup.find_all('div', {'class': 'job_seen_beacon'}):
                results.append(result)
            sleep(5)
        print(datetime.now())
        print("Gathered " + str(len(results)) + " jobs so far")
        print("---------- Completed for " + job)
        print("---------------------------------------------------------------")
    except Exception as e:
        print("ERROR: Something went wrong ", e)
        continue
    print("Scraping done !!! Check logs for more info")
end_time = datetime.now()
print(end_time)
print(end_time - start_time)

2022-05-17 23:06:28.788805
---------- Started for Data+Scientist
2022-05-17 23:14:00.273378
Gathered 2582 jobs so far
---------- Completed for Data+Scientist
---------------------------------------------------------------
Scraping done !!! Check logs for more info
---------- Started for full+stack+developer
2022-05-17 23:21:24.808264
Gathered 3766 jobs so far
---------- Completed for full+stack+developer
---------------------------------------------------------------
Scraping done !!! Check logs for more info
---------- Started for big+data+engineer
2022-05-17 23:28:50.852281
Gathered 5270 jobs so far
---------- Completed for big+data+engineer
---------------------------------------------------------------
Scraping done !!! Check logs for more info
---------- Started for software+architect
2022-05-17 23:36:20.403347
Gathered 6551 jobs so far
---------- Completed for software+architect
---------------------------------------------------------------
Scraping done !!! Check logs for more 

KeyboardInterrupt: 

In [38]:
df_jobs = pd.DataFrame(
    columns=['location', 'title', 'company', 'salary', 'summary', 'rating'])

In [39]:
for entry in results:
    location = get_loc(entry)
    title = get_job(entry)
    company = get_comp(entry)
    salary = get_sal(entry)
    desc = get_desc(entry)
    rating = get_rating(entry)
    df_jobs.loc[len(df_jobs)] = [
        location, title, company, salary, desc, rating
    ]

In [40]:
len(results)

12501

In [41]:
# df_jobs.drop_duplicates(inplace = True, keep='first')
# df_jobs
df_jobs.to_csv(r'./Dataset/job_scrap.csv')
df_jobs.describe()

,location,title,company,salary,summary,rating
count,12501,12501,12501,12501,12501,12501
unique,311,2375,1945,495,3745,32
top,"Bengaluru, Karnataka",DevOps Engineer,IBM,NA,\nSoftware Developers at IBM are the backbone ...,NA
freq,3173,521,1623,10216,559,5457


In [ ]:
df_jobs = pd.read_csv(r'./Dataset/job_raw.csv')
df_jobs.head()
df_jobs.isna().sum()

In [ ]:
df_jobs[df_jobs.duplicated()]
df_jobs.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
display(df_jobs.describe())
print(df_jobs.isna().sum())

In [ ]:
df_jobs.replace(to_replace='NA', value=np.nan, inplace=True)

In [ ]:
df = df_jobs

In [ ]:
print('Unique Locations  \t:', len(df.location.unique()))
print('Unique Title  \t\t:', len(df.title.unique()))
print('Unique Company  \t:', len(df.company.unique()))
print('Unique Salary \t\t:', len(df.salary.unique()))
print('Unique Rating \t\t:', len(df.rating.unique()))
# Unique Locations  	: 385
# Unique Title  		: 3409
# Unique Company  	: 2426
# Unique Salary 		: 605
# Unique Rating 		: 31

In [ ]:
# """
# * Noise found
#      1. 'Remote in '
#      2. '+1 location'
#      3. 'Hybrid remote in '
#      4. '+7 location'
#      5. '+2 location'
#      6. 'Temporarily Remote in '
# """

In [ ]:
stop_words = stopwords.words('english') + [
    'in', 'remote', 'Remote', 'location', 'locations', 'Temporarily', 'Hybrid',
    'India'
]
df['clean_location'] = df['location'].apply(lambda x: ' '.join(
    [word for word in x.split() if word not in (stop_words)]))

remove_list = ['+1', '+2', '+3', '+4', '+5', '+6', '+7', '+8', '+9', '9']
for i in remove_list:
    df.clean_location = df.clean_location.str.replace(i, '', regex=False)

df.clean_location = df.clean_location.str.strip()

In [ ]:
df.head()

In [ ]:
print('Uncleaned Locations  \t:', len(df.location.unique()))
print('Cleaned Locations  \t:', len(df.clean_location.unique()))

In [ ]:
# print_df(df.title.unique())

In [ ]:
stop_words = stopwords.words('english') +  ['Scien...', 'Forecastin...', '-', 'location', 'locations','Temporarily', 'Hybrid', 'India']
df['clean_title'] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
noise_char_title = ['new', '...', '-', '/', '(', ')', ':', '|', ',']
for i in noise_char_title:
    df.clean_title = df.clean_title.str.replace(i, '', regex=False)

In [ ]:
print_df(df.clean_title)

In [ ]:
for i in results:
    print(get_job(i))

In [ ]:
df['clean_title']

In [ ]:
df_comp_rat = df_jobs[['company', 'rating']]

In [ ]:
df_comp_rat.drop_duplicates(keep='first')

In [ ]:
import pandasql as psql

In [ ]:
df_temp = psql.sqldf("SELECT DISTINCT(clean_location) FROM df")

In [ ]:
df_clean_loc_list = df_temp.clean_location.str.split(',')
for i in df_clean_loc_list:
    if len(i) == 1:
        print(i)

In [ ]:
df_sal_list = df_jobs.salary.str.split(' ')
for i in df_sal_list:
    print(i)

In [ ]:
df_jobs